In [23]:
from theFuncs import *

In [24]:
# List of stocks (top 100 in S&P 500) and get returns
# returns of S&P500 (overall) -- use index returns to get return of market
# Time horizon: Last 2 years
# Get rolling betas (3 months)

In [25]:
# load_DTB3_SPY().to_csv('./data/spy_dtb3.csv')

In [26]:
sp100 = os.listdir(os.getcwd()+'/data/stock_dfs')
sp100_tickers = []
for csvfile in sp100:
    sp100_tickers.append(csvfile[:-4])

# sp100_tickers

In [27]:
ticker_data = get_ticker_data_multisource(sp100_tickers)

In [28]:
# ticker_data.loc['CSCO']

In [29]:
model = RollingOLS.from_formula(
        "(ret - DTB3) ~ (spy_ret - DTB3)", 
        data = ticker_data.loc[sp100_tickers[0]], 
        window = 66
    ).fit(params_only=True)
df = model.params
df.columns = ["alpha", "Beta"]
df['ticker'] = sp100_tickers[0]
df.set_index('ticker', inplace=True, append=True)
df = df.reorder_levels(['ticker', 'date']).sort_index()


for ticker in sp100_tickers[1:]:
    model = RollingOLS.from_formula(
        "(ret - DTB3) ~ (spy_ret - DTB3)", 
        data = ticker_data.loc[ticker], 
        window = 66
    ).fit(params_only=True)

    temp = model.params
    temp.columns = ["alpha", "Beta"]
    temp['ticker'] = ticker
    temp.set_index('ticker', inplace=True, append=True)
    temp = temp.reorder_levels(['ticker', 'date']).sort_index()
    df = pd.concat(
        [ 
            df, 
            temp
        ]
    )

ticker_data = ticker_data.join(df)

ticker_data

high        low       open      close       volume  \
ticker date                                                                  
AAPL   2019-01-03  36.430000  35.500000  35.994999  35.547501  365248800.0   
       2019-01-04  37.137501  35.950001  36.132500  37.064999  234428400.0   
       2019-01-07  37.207500  36.474998  37.174999  36.982498  219111200.0   
       2019-01-08  37.955002  37.130001  37.389999  37.687500  164101200.0   
       2019-01-09  38.632500  37.407501  37.822498  38.327499  180396400.0   
...                      ...        ...        ...        ...          ...   
XOM    2021-10-26  65.940002  64.360001  64.489998  65.839996   25585100.0   
       2021-10-27  65.250000  63.980000  65.160004  64.129997   21764500.0   
       2021-10-28  64.470001  63.630001  63.820000  64.309998   16626400.0   
       2021-10-29  65.360001  64.330002  65.050003  64.470001   23963300.0   
       2021-11-01  66.080002  65.019997  65.070000  65.629997   19656400.0   

                        ret   spy_close    DTB3   spy_ret     alpha      Beta  
ticker date                                                                    
AAPL   2019-01-03 -0.099607  232.998627  0.0236 -0.023863       NaN       NaN  
       2019-01-04  0.042689  240.803085  0.0237  0.033496       NaN       NaN  
       2019-01-07 -0.002226  242.701736  0.0241  0.007885       NaN       NaN  
       2019-01-08  0.019063  244.981995  0.0241  0.009395       NaN       NaN  
       2019-01-09  0.016982  246.126907  0.0240  0.004673       NaN       NaN  
...                     ...         ...     ...       ...       ...       ...  
XOM    2021-10-26  0.023155  455.959991  0.0006  0.000900  0.001583  1.195824  
       2021-10-27 -0.025972  453.940002  0.0006 -0.004430  0.000920  1.218142  
       2021-10-28  0.002807  458.320007  0.0006  0.009649  0.000897  1.179621  
       2021-10-29  0.002488  459.250000  0.0005  0.002029  0.000787  1.181637  
       2021-11-01  0.017993  460.040009  0.0005  0.001720  0.000920  1.179371  

[72061 rows x 11 columns]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=80cc924b-f00e-4408-bf29-ac16ba2efaff' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>